<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 
<h2>10</h2>


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

<p> <b>Описание задачи:</b>
<p>Создать базовый класс Invoice в C#, который будет представлять информацию о
фактурах за поставленные товары или оказанные услуги. На основе этого класса
разработать 2-3 производных класса, демонстрирующих принципы наследования и
полиморфизма. В каждом из классов должны быть реализованы новые атрибуты и
методы, а также переопределены некоторые методы базового класса для
демонстрации полиморфизма.</p>
<p><b>Требования к базовому классу Invoice:</b>
<ul>
 <li><b>Атрибуты:</b> Номер фактуры (InvoiceNumber), Дата выдачи (IssueDate), Общая сумма (TotalAmount).</li>
 <li><b>Методы:</b>
    <ul>
        <li>CalculateTotal(): метод для расчета общей суммы по фактуре. </li>
        <li>AddLine(LineItem lineItem): метод для добавления позиции в фактуру.</li>
        <li>RemoveLine(LineItem lineItem): метод для удаления позиции из
фактуры.</li>
    </ul>
  </li>
  </ul>
  </p>

<p><b>Требования к производным классам: </b> </p>

<p>
<ol>
 <li>ТоварнаяФактура (GoodsInvoice): Должна содержать дополнительные
атрибуты, такие как Дата поставки (SupplyDate). Метод AddLine() должен
быть переопределен для добавления информации о дате поставки товара
при добавлении позиции. </li>
 <li>УслуговаяФактура (ServiceInvoice): Должна содержать дополнительные
атрибуты, такие как Дата оказания услуги (ServiceDate).
Метод RemoveLine() должен быть переопределен для добавления
информации о причине аннулирования услуги при удалении позиции. </li>
  <li>КомбинированнаяФактура (CombinedInvoice) (если требуется третий класс):
Должна содержать дополнительные атрибуты, такие как Наличие возврата
(ReturnAllowed). Метод CalculateTotal() должен быть переопределен для
учета возможного возврата товара или услуги при расчете общей суммы. </li> 
 </ol>
</p>


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;

//Базовый класс для создания объекта фактуры
public class LineItem{
    public string Description { get; set; }
    public decimal UnitPrice { get; set; }
    public int Quantity { get; set; }

    public decimal Total => UnitPrice * Quantity;
    
    public LineItem(string description, decimal unitPrice, int quantity)
    {
        Description = description;
        UnitPrice = unitPrice;
        Quantity = quantity;
    }
}

//Класс для создания объекта товарной фактуры
public class GoodsLineItem : LineItem
{
    public DateTime SupplyDate { get; set; }


    public GoodsLineItem(string description, decimal unitPrice, int quantity, DateTime supplyDate)
        : base(description, unitPrice, quantity)
    {
        SupplyDate = supplyDate;
    }

}

//Класс для создания объекта сервисной фактуры
public class ServiceLineItem : LineItem
{
    public DateTime ServiceDate { get; set; }


    public ServiceLineItem(string description, decimal unitPrice, int quantity, DateTime serviceDate)
        : base(description, unitPrice, quantity)
    {
        ServiceDate = serviceDate;
    }

}

//Класс для создания объекта комбинированной фактуры
public class CombinedLineItem : LineItem
{
    public bool ReturnAllowed { get; set; }


    public CombinedLineItem(string description, decimal unitPrice, int quantity, bool returnAllowed)
        : base(description, unitPrice, quantity)
    {
        ReturnAllowed = returnAllowed;
    }

}

//Абстрактный класс фактур в целом
public abstract class Invoice{
    public string InvoiceNumber { get; protected set;}
    public DateTime IssueDate {get; protected set;}
    public decimal TotalAmount => CalculateTotal();
    public IReadOnlyList<LineItem> LineItems => _lineItems.AsReadOnly();


    private readonly List<LineItem> _lineItems = new List<LineItem>();


    protected Invoice(string invoiceNumber, DateTime issueDate)
    {
        InvoiceNumber = invoiceNumber;
        IssueDate = issueDate;
    }

    public virtual decimal CalculateTotal()
    {
        return _lineItems.Sum(item => item.Total);
    }

    public virtual void AddLine(LineItem lineItem)
    {
        if (lineItem == null)
            throw new ArgumentNullException(nameof(lineItem));
            
        _lineItems.Add(lineItem);
    }

    public virtual void RemoveLine(LineItem lineItem)
    {
        if (lineItem == null)
            throw new ArgumentNullException(nameof(lineItem));
            
        _lineItems.Remove(lineItem);
    }

    public abstract string GetInvoiceType();


}


//Класс товарной фактуры
class GoodsInvoice: Invoice{
    public DateTime SupplyDate {get; protected set;}

    public GoodsInvoice(string invoiceNumber, DateTime issueDate, DateTime supplyDate) 
            : base(invoiceNumber, issueDate)
    {
        SupplyDate = supplyDate;
    }

    public override void AddLine(LineItem lineItem)
    {
        if (lineItem is GoodsLineItem goodsLine)
        {
            base.AddLine(goodsLine);
        }
        else
        {
            throw new ArgumentException("Товарная фактура поддерживает только тип GoodsLineItem");
        }
    }

    public override string GetInvoiceType()
    {
        return "Товарная фактура";
    }
}


//Интерфейс сервисной фактуры с причиной отмены услуги
public interface IServiceInvoice
{
    DateTime ServiceDate { get; set; }
    void RemoveLine(LineItem lineItem, string cancellationReason);
}


//Класс сервисной фактуры
class ServiceInvoice: Invoice, IServiceInvoice{
    public DateTime ServiceDate {get; set;}

    public ServiceInvoice(string invoiceNumber, DateTime issueDate, DateTime serviceDate) 
            : base(invoiceNumber, issueDate)
    {
        ServiceDate = serviceDate;
    }

    public override void RemoveLine(LineItem lineItem)
    {
        RemoveLine(lineItem, "Причина не указана");
    }

    public void RemoveLine(LineItem lineItem, string cancellationReason)
    {
        base.RemoveLine(lineItem);
        Console.WriteLine($"Услуга аннулирована. Причина: {cancellationReason}");
    }

    public override string GetInvoiceType()
    {
        return "Сервисная фактура";
    }
}

//Класс комбинированной фактуры
class CombinedInvoice: Invoice{
    public bool ReturnAllowed { get; set; }
    private decimal _returnAmount;

    public CombinedInvoice(string invoiceNumber, DateTime issueDate, bool returnAllowed = false) 
            : base(invoiceNumber, issueDate)
    {
        ReturnAllowed = returnAllowed;
    }

    public void SetReturnAmount(decimal amount)
        {
            if (!ReturnAllowed)
                throw new InvalidOperationException("Возврат не разрешен для этой фактуры");
            
            _returnAmount = amount;
        }

    public decimal CalculateReturn()
    {
        return ReturnAllowed ? _returnAmount : 0;
    }

    public override decimal CalculateTotal(){
        var total = base.CalculateTotal();
        return total - CalculateReturn();
    }

    public override string GetInvoiceType()
    {
        return "Комбинированная фактура";
    }
}

//Класс для создания фактур (паттерн фабрика)
public static class InvoiceFactory
    {
        public static Invoice CreateGoodsInvoice(string number, DateTime issueDate, DateTime supplyDate)
        {
            return new GoodsInvoice(number, issueDate, supplyDate);
        }

        public static Invoice CreateServiceInvoice(string number, DateTime issueDate, DateTime serviceDate)
        {
            return new ServiceInvoice(number, issueDate, serviceDate);
        }

        public static Invoice CreateCombinedInvoice(string number, DateTime issueDate, bool returnAllowed)
        {
            return new CombinedInvoice(number, issueDate, returnAllowed);
        }
    }

// Демонстрация использования
class Program
    {
        static void Main(string[] args)
        {
            var goodsInvoice = InvoiceFactory.CreateGoodsInvoice(
                "INV-001", 
                DateTime.Now, 
                DateTime.Now.AddDays(7));
            
            var serviceInvoice = InvoiceFactory.CreateServiceInvoice(
                "INV-002",
                DateTime.Now,
                DateTime.Now.AddDays(3));
            
            var combinedInvoice = InvoiceFactory.CreateCombinedInvoice(
                "INV-003",
                DateTime.Now,
                true);

            var product = new GoodsLineItem("Ноутбук", 1500m, 2, DateTime.Now.AddDays(7));
            var service = new ServiceLineItem("Техническое обслуживание", 300m, 1, DateTime.Now.AddDays(3));

            goodsInvoice.AddLine(product);
            serviceInvoice.AddLine(service);

            ((CombinedInvoice)combinedInvoice).SetReturnAmount(200m);
            combinedInvoice.AddLine(new LineItem("Планшет", 800m, 1));

            var invoices = new List<Invoice> { goodsInvoice, serviceInvoice, combinedInvoice };

            foreach (var invoice in invoices)
            {
                Console.WriteLine($"\n{invoice.GetInvoiceType()}:");
                Console.WriteLine($"Общая сумма: {invoice.CalculateTotal():C}");
                Console.WriteLine($"Количество позиций: {invoice.LineItems.Count}");
            }

            ((ServiceInvoice)serviceInvoice).RemoveLine(service, "Клиент отказался от услуги");
        }
    }
